In [95]:
import pandas as pd
import numpy as np
import time
import math
import datetime
import statistics
import random
import heapq
from scipy import optimize
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from scipy.ndimage import uniform_filter1d

# data = pd.read_csv('/Users/windyluan/Dropbox/VIS/Insight/Data/carSales2.csv')
data = pd.read_csv('C:\\Users\\52497\\Dropbox\\VIS\\Insight\\Data\\carSales1.csv')
# data = pd.read_csv('C:\\Users\\52497\\Dropbox\\VIS\\Insight\\Data\\carSales2.csv')
# data = pd.read_csv('C:\\Users\\52497\\Dropbox\\VIS\\FactExtraction\\Data\\Emission.csv')
# data = pd.read_csv('C:\\Users\\52497\\Dropbox\\VIS\\Insight\\Data\\aggregation.csv')
# data = pd.read_csv('C:\\Users\\52497\\Dropbox\\VIS\\FactExtraction\\Data\\Census.csv')

# data['Year'] = data['Year'].apply(lambda year_string: year_string.split('/')[0])
# data['Year'] = data['Year'].apply(lambda time_str: time.mktime(datetime.datetime.strptime(time_str, "%Y/%d/%m").timetuple()))

data = data.loc[(data!=0).any(axis=1)].dropna()

In [96]:
def subspace_ordering(feature_measures, df):
    feature_unique_value_matrix = dict(zip(feature_measures,
                                       [df[feature].nunique() for feature in feature_measures]))
    sorted_feature_dict = {k: v for k, v in sorted(feature_unique_value_matrix.items(), key=lambda item: item[1], reverse=False)}
    print("New Keys: ", list(sorted_feature_dict.keys()))
    print(sorted_feature_dict)
    return list(sorted_feature_dict.keys())

In [97]:
data

,Year,Brand,Category,Sales
0,2007/1/1,BMW,Compact,142490
1,2008/1/1,BMW,Compact,112464
2,2009/1/1,BMW,Compact,90960
3,2010/1/1,BMW,Compact,100910
4,2011/1/1,BMW,Compact,94371
...,...,...,...,...
270,2007/1/1,Volkswagen,SUV,8812
271,2008/1/1,Volkswagen,SUV,6755
272,2009/1/1,Volkswagen,SUV,4392
273,2010/1/1,Volkswagen,SUV,4713


In [98]:
def point_power_law(phi):
    """
    :param: phi
    :return: [breakdown_value, observation_value, predict_value]
    """
    ordered_phi = {k: v for k, v in sorted(phi.items(), key=lambda item: item[1], reverse=True)}
    keys = list(ordered_phi.keys())
    values = list(ordered_phi.values())
    max_value = max(values)
    ydata = []
    for i in values:
        if i != max_value and i not in ydata:
            ydata.append(i / max_value)
    xdata = range(2, len(ydata) + 2)
    logx = np.log10(xdata)
    logy = np.log10(ydata)
    pinit = [1.0, -1.0]

    fitfunc = lambda p, x: p[0] + p[1] * x
    errfunc = lambda p, x, y: (y - fitfunc(p, x))
    powerLawFunc = lambda amp, index, x: amp * (x ** index)

    try:
        out = optimize.leastsq(errfunc, pinit, args=(logx, logy), full_output=1)

        pfinal = out[0]
        covar = out[1]

        index = pfinal[1]
        amp = 10.0 ** pfinal[0]
        # indexErr = np.sqrt(covar[1][1])
        # ampErr = np.sqrt(covar[0][0]) * amp


        # predict_errs = ([1] + ydata) - powerLawFunc(amp, index, range(1, len(ydata) + 2))
        # print(predict_errs)
        # mu, std = norm.fit(predict_errs)
        # plt.hist(predict_errs, density=True, alpha=0.6, color='g')
        # plt_xmin, plt_xmax = plt.xlim()
        # plt_x = np.linspace(plt_xmin, plt_xmax, 100)
        # p = norm.pdf(plt_x, mu, std)
        # plt.plot(plt_x, p, 'k', linewidth=2)
        # title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
        # plt.title(title)
        # plt.show()

        sig = norm.cdf(1 - powerLawFunc(amp, index, 1), 0, 0.3)
        # print(predict_err, sig)
        #  [breakdown_value, sig]
        return [keys[0], sig]

    except TypeError:
        return [keys[0], -1]

In [99]:
def calc_top1_insight(subspace_df, breakdown, measure):
    global sum_impact
    breakdown_measure_dict = dict(zip(list(subspace_df[breakdown]),list(subspace_df[measure])))
    res = point_power_law(breakdown_measure_dict)
    # result_dict = dict(zip(
    #     ['breakdown','breakdown_value', 'measure', 'predict', 'abs_impact', 'insight', 'insight_type', 'score'],
    #     [breakdown] + res + [absolute_impact, 'top1', 'point', random.random()]))
    result_dict = dict(zip(
    ['breakdown','breakdown_value', 'sig'],
    [breakdown] + res ))
    return result_dict

def get_subspace_df(subspace_condition_dict, df):
    condition = pd.Series(True, index=df.index)
    for feature in subspace_condition_dict:
        if subspace_condition_dict[feature] == '*':
            continue
        condition = condition & (df[feature] == subspace_condition_dict[feature])
    return df[condition]

def calc_top1_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0:
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    calc_dict = calc_top1_insight(grouped_df, breakdown, measure)
    res_dict = dict(zip(['impact', 'insight', 'insight_type', 'score'],
                        [impact, 'top1', 'point', calc_dict.get('sig')*impact]))
    top1_insight = dict(subspace_condition_dict, **calc_dict, **res_dict)
    return top1_insight

def calc_trend_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global time_col
    global sum_impact
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    #  Calculate trend insight
    x = grouped_df[breakdown].values #
    y = grouped_df[measure].values #
    x = x.reshape(-1, 1) #
    y = y.reshape(-1, 1) #
    if x.shape[0] < 2: return None
    reg = LinearRegression().fit(x, y) #
    slope = reg.coef_[0][0] #
    r2_score = reg.score(x, y)
    # sig = r2_score * norm.cdf(abs(slope), 0.2, 10000)
    sig = r2_score
    # grouped_df.plot(x=breakdown, y=measure)
    # plt.plot(grouped_df[breakdown], reg.predict(x))
    # plt.title("Sig Score is: {}, subspace condition: {}".format(sig, subspace_condition_dict))
    # plt.show()
    # print(subspace_condition_dict, "Sig: ", sig, "Impact: ", impact)
    result_dict = dict(zip(['breakdown','breakdown_value', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, -1] + [sig, impact] + ['trend', 'shape', sig*impact]))
    trend_insight = dict(subspace_condition_dict, **result_dict)
    return trend_insight

def calc_change_point_abs(y, y_MA):
    y_change = []
    for index in range(len(y)):
        if index == 0 or index == len(y) - 1:
            y_change.append(0)
        elif 1 <= index < len(y) - 2:
            y_change.append(abs(y_MA[index-1] - y_MA[index+2]))
        elif index == len(y) - 2:
            y_change.append(abs(y_MA[index-1] - y[index+1]))
    return y_change

def calc_change_point_sigma(y, y_MA, y_MA_square):
    y_sigma = []
    for index in range(len(y)):
        if index == 0 or index == len(y) - 1:
            y_sigma.append(0)
        elif 1 <= index < len(y) - 2:
            res = np.sqrt((y_MA_square[index-1] + y_MA_square[index+2])/2 - np.square(y_MA[index-1] + y_MA[index+2]) / 4)
            y_sigma.append(res)
        elif index == len(y) - 2:
            res = np.sqrt((y_MA_square[index-1] + y[index+1]**2)/2 - np.square(y_MA[index-1] + y[index+1]) / 4)
            y_sigma.append(res)
    return y_sigma

def calc_change_point_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    MA_size = 2
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if len(y) < 4:
        return None
    y_MA = uniform_filter1d(y, size=MA_size, mode='nearest')
    y_MA_square = uniform_filter1d(np.square(y), size=MA_size, mode='nearest')
    y_change = calc_change_point_abs(y, y_MA)
    y_sigma = calc_change_point_sigma(y=y, y_MA=y_MA, y_MA_square=y_MA_square)
    k_mean = [y_change[i] / y_sigma[i] if y_sigma[i]!=0 else 0 for i in range(1, len(y) - 1)]
    k_mean_max = max(k_mean)
    sig = norm.cdf(k_mean_max * np.square(MA_size))
    breakdown_value = x[k_mean.index(k_mean_max) + 1]
    # print("The significance is {}, with value = {}".format(sig, breakdown_value))
    result_dict = dict(zip(['breakdown','breakdown_value', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, breakdown_value] + [sig, impact] + ['change point', 'shape', sig*impact]))
    change_point_insight = dict(subspace_condition_dict, **result_dict)
    if not sig:
        return None
    return change_point_insight

def calc_outlier_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if len(y) < 4:
        return None
    mean = np.mean(y)
    std = np.std(y)
    z_score = (y - mean) / std
    z_score_max_index = np.argmax(z_score)
    z_score_max = z_score[z_score_max_index]
    sig = norm.cdf(z_score_max)
    breakdown_value = x[z_score_max_index]
    if math.isnan(sig) :
        return None
    # print("The significance is {}, with value = {}, kmeans is {}".format(sig, breakdown_value, z_score_max))
    result_dict = dict(zip(['breakdown','breakdown_value', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, breakdown_value] + [sig, impact] + ['outlier', 'shape', sig*impact]))
    outlier_insight = dict(subspace_condition_dict, **result_dict)

    return outlier_insight

def calc_attribution_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0:
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if np.sum(y) == 0:
        return None
    portion = np.max(y)/np.sum(y)

    if portion < 0.5:
        return None

    sig = norm.cdf(portion, 0.5, 0.3)
    breakdown_index = np.argmax(y)

    result_dict = dict(zip(['breakdown','breakdown_value', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, x[breakdown_index]] + [sig, impact] + ['attribution', 'point', sig*impact]))
    attribution_insight = dict(subspace_condition_dict, **result_dict)
    return attribution_insight

def calc_evenness_insights(subspace_condition_dict, breakdown, measure, subspace_df, impact):
    global sum_impact
    if subspace_df.shape[0] == 0 or breakdown != time_col or subspace_condition_dict[time_col] != '*':
        return None
    grouped_df = subspace_df.groupby(breakdown, as_index=False).sum()
    x = grouped_df[breakdown].values
    y = grouped_df[measure].values
    if len(y) == 1: return None
    std = np.std(y)
    # print(std)
    sig = 2 * (1 - norm.cdf(std, 0, 3))
    result_dict = dict(zip(['breakdown','breakdown_value', 'sig', 'impact', 'insight', 'insight_type',
                            'score'],
                           [breakdown, -1] + [sig, impact] + ['evenness', 'point', sig*impact]))
    evenness_insight = dict(subspace_condition_dict, **result_dict)
    return evenness_insight

In [100]:

def generate_array(arr, index, val):
    new_arr = arr[:]
    new_arr[index] = val
    return new_arr

def generate_process_node(feature_names, measure, output_array, df,
                          calc_insights = lambda: "Need to define", breakdowns = [], top_num = 50):
    def process_node(node):
        star_feature_name = [feature_names[index] for index in range(len(node)) if node[index] == '*']
        current_breakdowns = star_feature_name if len(breakdowns) == 0 else breakdowns
        for breakdown in current_breakdowns:
            # subspace_condition_dict. e.g., ['Year', 'Brand', 'Category'] ['2007', '*', '*']
            subspace_df = get_subspace_df(dict(zip(feature_names, node)), df)
            impact = subspace_df[measure].count() / sum_impact
            subspace_condition_dict = dict(zip(feature_names, node))

            if len(output_array) == top_num and output_array[0][0] > impact:
                return False

            insight = calc_insights(subspace_condition_dict=subspace_condition_dict,
                         breakdown=breakdown, measure=measure, impact = impact, subspace_df=subspace_df)
            if insight:
                # output.append(insight)
                if len(output_array) < top_num:
                    heapq.heappush(output_array, (insight.get('score'), random.random(), insight))
                elif output_array[0][0] < insight.get('score'):
                    heapq.heapreplace(output_array, (insight.get('score'), random.random(), insight))
        if node.count('*') == 1:
            return False
        return True
    return process_node

def BFS_tranverse_and_process(df, feature_names, process_node = lambda x: True):
    feature_unique_value_matrix = [df[feature].unique().tolist() for feature in feature_names]
    traverse_root = ['*' for i in feature_names] + [[]]
    result_stack = [traverse_root] 
    
    while len(result_stack) > 0:
        *root, ban = result_stack.pop(0)
        result = process_node(root)
        if not result:
            continue
        banned_index = ban[:]
        for col_index, val_index in enumerate(root):
            if col_index in banned_index:
                continue
            banned_index.append(col_index)
            banned_index = list(set(banned_index))
            if val_index == '*':
                result_stack += ([generate_array(root, col_index, val)+[banned_index[:]]  for val in feature_unique_value_matrix[col_index]])

In [101]:
start_time = time.time()
output1= []
output2= []
output3= []
output4= []
output5= []

# CarSales1.csv
feature_names = ['Year', 'Brand', 'Category']
measure = 'Sales'
time_col = 'Year'
data[time_col] = data[time_col].apply(lambda year_string: year_string.split('/')[0])


# carSales2.csv
# feature_names = ['Year', 'Brand', 'Body', 'Engine Type', 'EngineV', 'Model', 'Registration']
# measure = 'Price'
# time_col = 'Year'


# Census.csv
# feature_names = ['Birthday', 'Age Segment', 'Marital Status', 'Sex', 'Age Group']
# measure = 'Count of persons'
# time_col = 'Birthday'
#
# data[time_col] = data[time_col].apply(lambda year_string: year_string.split('/')[0])


# Emission.csv
# feature_names = ['Year', 'State', 'Producer Type', 'Energy Source']
# measure = 'CO2 (kt)'
# time_col = 'Year'


sum_impact = data[measure].count()
feature_names = subspace_ordering(feature_measures=feature_names, df=data)

process_node_trend = generate_process_node(feature_names = feature_names,
                                     output_array=output1,
                                     df = data,
                                     measure = measure,
                                     calc_insights=calc_trend_insights)

BFS_tranverse_and_process(data,feature_names, process_node=process_node_trend)

process_node_top1 = generate_process_node(feature_names = feature_names,
                                     output_array=output2,
                                     df = data,
                                     measure = measure,
                                     calc_insights=calc_top1_insights)

BFS_tranverse_and_process(data,feature_names, process_node=process_node_top1)

process_node_change_point = generate_process_node(feature_names = feature_names,
                                     output_array=output3,
                                     df = data,
                                     measure = measure,
                                     calc_insights=calc_change_point_insights)

BFS_tranverse_and_process(data,feature_names, process_node=process_node_change_point)


process_node_outlier = generate_process_node(feature_names = feature_names,
                                     output_array=output4,
                                     df = data,
                                     measure = measure,
                                     calc_insights=calc_outlier_insights)

BFS_tranverse_and_process(data,feature_names, process_node=process_node_outlier)
#
process_node_attribution = generate_process_node(feature_names = feature_names,
                                     output_array=output5,
                                     df = data,
                                     measure = measure,
                                     calc_insights=calc_attribution_insights)

BFS_tranverse_and_process(data,feature_names, process_node=process_node_attribution)

# process_node_evenness = generate_process_node(feature_names = feature_names,
#                                      output_array=output,
#                                      df = data,
#                                      measure = measure,
#                                      calc_insights=calc_evenness_insights)
#
# BFS_tranverse_and_process(data,feature_names, process_node=process_node_evenness)

res = [out[-1] for out in output1]\
      + [out[-1] for out in output2]\
      + [out[-1] for out in output3]\
      + [out[-1] for out in output4]\
      + [out[-1] for out in output5]



df = pd.DataFrame(res)
df['measure'] = measure
end_time = time.time()
print("Time Elapsed: {}".format(end_time-start_time ))

New Keys:  ['Year', 'Brand', 'Category']
{'Year': 5, 'Brand': 8, 'Category': 8}
Time Elapsed: 1.3813107013702393


In [102]:
df

,Year,Brand,Category,breakdown,breakdown_value,sig,impact,insight,insight_type,score,measure
0,*,Mazda,SUV,Year,-1,0.038017,0.018182,trend,shape,0.000691,Sales
1,*,BMW,SUV,Year,-1,0.015964,0.054545,trend,shape,0.000871,Sales
2,*,*,SUV,Year,-1,0.002184,0.400000,trend,shape,0.000873,Sales
3,*,GMC,*,Year,-1,0.047839,0.109091,trend,shape,0.005219,Sales
4,*,Mazda,MPV,Year,-1,0.049890,0.018182,trend,shape,0.000907,Sales
...,...,...,...,...,...,...,...,...,...,...,...
232,2011,GMC,*,Category,SUV,0.608955,0.021818,attribution,point,0.013286,Sales
233,*,*,MPV,Brand,Mazda,0.952210,0.018182,attribution,point,0.017313,Sales
234,2011,*,Sporty,Brand,Ford,0.860858,0.014545,attribution,point,0.012522,Sales
235,*,Mazda,*,Category,Compact,0.534577,0.090909,attribution,point,0.048598,Sales


In [103]:
print("Time Elapsed: {}".format(end_time-start_time ))

Time Elapsed: 1.3813107013702393


In [104]:
"""
print point insight result
"""
# for insight in res:
#     # breakdown_cond = {insight['breakdown']: insight['breakdown_value']}
#     subspace_cond = {k:insight[k] for k in insight.keys() if k in feature_names}
#     condition = dict(subspace_cond)
#     selected_df = get_subspace_df(condition, data)
#     grouped_df = selected_df.groupby(insight['breakdown'], as_index=False).sum()
#     if insight['insight'] == 'top1':
#         grouped_df = grouped_df.sort_values(by=[measure])
#         grouped_df.plot(x=insight['breakdown'], y=measure, kind='bar')
#     elif insight['insight'] == 'trend' or insight['insight'] == 'evenness':
#         # print(grouped_df)
#         grouped_df = grouped_df.sort_values(by=time_col)
#         grouped_df.plot(x=insight['breakdown'], y=measure, kind='line')
#     elif insight['insight'] == 'change point' or insight['insight'] == 'outlier':
#         grouped_df = grouped_df.sort_values(by=time_col)
#         y_val = grouped_df.loc[grouped_df[insight['breakdown']] == insight['breakdown_value']][measure]
#         # print(insight['breakdown_value'], y_val.values[0])
#         plt.plot(grouped_df[insight['breakdown']], grouped_df[measure])
#         plt.plot([insight['breakdown_value']], [y_val.values[0]],color='red', marker='o', markersize=3)
#     elif insight['insight'] == 'attribution':
#         grouped_df = grouped_df.sort_values(by=[measure])
#         plt.pie(grouped_df[measure], labels = grouped_df[insight['breakdown']], startangle = 90,
#         counterclock = False)
#         plt.axis('square')
#
#     plt.title(condition)
#     plt.show()

'\nprint point insight result\n'

In [105]:
with open('C:\\Users\\52497\\Dropbox\\VIS\\Insight\\res\\insights_carSales1.csv', 'w+') as file:
    df.to_csv(file, index=False, line_terminator='\n')
